In [ ]:
import numpy as np
import os
import pandas as pd
import plotly.express as px

In [ ]:
%run ../tasking_manager_stats/data_management
%run ../tasking_manager_stats/map_tools

In [ ]:
if os.name == "nt":
    encoding = 'ANSI' # Windows
else:
    encoding = 'utf-8' # Linux
print(f"Chosen encoding : {encoding}")

## Load data

In [ ]:
df = pd.read_csv(os.path.join(get_data_dir(), 'merged_stats.csv'), encoding=encoding)
df.head()

## Filter on CartONG projects


<font color='red'>WARNING: File to update</font>

The next section can be runned without it but we count data from projects where CartONG intervene only in a mapathon.

In [ ]:
raw_projects = pd.read_csv(os.path.join(get_data_dir(), 'Orga_Missing_Maps_CartONG_2022_04_24.csv'), header=1)['N° Projet']
projects = []
for s in raw_projects:
    try:
        projects.append(int(s))
    except:
        pass
df = df[df['Project'].isin(projects)]

## Filter on a year

In [ ]:
year = 2021

In [ ]:
os.makedirs(str(year), exist_ok=True)

In [ ]:
df = df[df["Year"] == year]

In [ ]:
df.to_csv(os.path.join(str(year), "raw_data.csv"), index=None, encoding=encoding)

## Remove a week of Nuit de la Géo

In [ ]:
# ndg_month = 4
# ndg_day = 8
# df = df[(df['Month'] != ndg_month) | (~df['Day'].isin(np.arange(ndg_day, ndg_day + 8)))]

## New contributors by day

In [ ]:
date_cols = ["Year", "Month", "Day"]
df['Date'] = pd.to_datetime(df[date_cols].apply(lambda x: '-'.join(x.values.astype(str)), axis="columns"))
df.head()

In [ ]:
author_first_date = df[["Author", "Date"]].groupby("Author").min()
author_first_date.head()

In [ ]:
contributors_first_date = author_first_date.reset_index().groupby('Date').count().cumsum()
contributors_first_date.head()

In [ ]:
fig = px.line(contributors_first_date.reset_index(), x="Date", y="Author")
fig.update_xaxes(title_text = f'Date of first contribution in {year}')
fig.update_yaxes(title_text = "Contributors number")
fig.update_layout(title={'text':f'Cumulative sum of contributors in {year}'})

## Duration by date

In [ ]:
df["Duration_h"] = df["Duration"] / 3600

In [ ]:
fig = px.bar(df[["Month", "Duration_h"]].groupby("Month").sum().reset_index(), x="Month", y="Duration_h")
fig.update_xaxes(title_text = 'Month')
fig.update_yaxes(title_text = "Contribution time in hours")
fig.update_layout(title={'text':f'Contribution time by month in {year}'})

In [ ]:
df2 = df[["Month", "Day", "Hour", "Duration_h"]].copy()
# Convert UTC to FR-Paris
df2['Hour'] += 1
df2.loc[(df2["Month"] < 11) & (df2["Month"] > 3), "Hour"] += 1
# Offset for better graph
df2['Hour'] -= 0.5
fig = px.bar(df2.groupby("Hour").sum().reset_index(), x="Hour", y="Duration_h")
fig.update_xaxes(title_text = 'Hour of the Day')
fig.update_yaxes(title_text = "Contribution time in hours")
fig.update_layout(title={'text':f'Contribution time by hour of the day in {year}'})

In [ ]:
df["DayOfWeek"] = df["Date"].apply(lambda dt: dt.weekday())
df.head()

In [ ]:
fig = px.bar(df[["DayOfWeek", "Duration_h"]].groupby("DayOfWeek").sum().reset_index(), x="DayOfWeek", y="Duration_h")
fig.update_xaxes(title_text = 'DayOfWeek')
fig.update_yaxes(title_text = "Contribution time in hours")
fig.update_layout(title={'text':f'Contribution time by day of week in {year}'})
fig.update_xaxes(tickmode="array",
                 tickvals=np.arange(7),
                 ticktext=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"])

## Contributors by project

In [ ]:
df_pro_user = df[['Project', 'Author']].drop_duplicates()
df_pro_user.head()

In [ ]:
g = df_pro_user.groupby('Author').count()
g[g['Project'] >= 10].sort_values('Project', ascending=False)

In [ ]:
more_than_three_projects = (g['Project'] >= 3).sum()
print(f"{len(g)} contributors in {year}")
print(f"{more_than_three_projects} contributors on 3 projects or more in {year} ({more_than_three_projects/len(g):.0%}) ")

In [ ]:
g.sort_values('Project', ascending=False).reset_index().to_csv(os.path.join(str(year), "Contributor_by_project_number.csv"), index=None, encoding=encoding)

### Restriction to mapping

In [ ]:
df_pro_user_map = df[df['Type'] == 'MAPPING'][['Project', 'Author']].drop_duplicates()
df_pro_user_map.head()

In [ ]:
g_map = df_pro_user_map.groupby('Author').count()
g_map[g_map['Project'] > 1].sort_values('Project', ascending=False).head(10)

In [ ]:
map_more_than_three_projects = (g_map['Project'] >= 3).sum()
print(f"{len(g_map)} mapping contributors in {year}")
print(f"{map_more_than_three_projects} mapping contributors on 3 projects or more in {year} ({map_more_than_three_projects/len(g_map):.0%}) ")

### Restriction to validation

In [ ]:
df_pro_user_valid = df[df['Type'] == 'VALIDATION'][['Project', 'Author']].drop_duplicates()
df_pro_user_valid.head()

In [ ]:
g_valid = df_pro_user_valid.groupby('Author').count()
g_valid[g_valid['Project'] > 1].sort_values('Project', ascending=False).head(10)

In [ ]:
valid_more_than_three_projects = (g_valid['Project'] >= 3).sum()
print(f"{len(g_valid)} validators in {year}")
print(f"{valid_more_than_three_projects} validators on 3 projects or more in {year} ({valid_more_than_three_projects/len(g_valid):.0%}) ")

In [ ]:
g_valid.sort_values('Project', ascending=False).reset_index().to_csv(os.path.join(str(year), "Validator_by_project_number.csv"), index=None, encoding=encoding)

### Ideas  :
- Keep only people who have participated to a mapathon
- Compute the number of mapathons by people
- Compute the number of contributions after a mapathons (time, tasks, projects)

## Number of day contribution

In [ ]:
df_day = df[["Author", "Year", "Month", "Day"]].drop_duplicates()[["Author","Day"]].groupby("Author").count()
df_day[df_day["Day"] > 30].sort_values("Day", ascending=False)

In [ ]:
contrib_5_days = (df_day['Day'] >= 5).sum()
print(f"{contrib_5_days} contributors on 5 days or more in {year} ({contrib_5_days/len(df_day):.1%})")

In [ ]:
df_day_map = df[df["Type"] == "MAPPING"][["Author", "Year", "Month", "Day"]].drop_duplicates()[["Author","Day"]].groupby("Author").count()
df_day_map[df_day_map["Day"] > 30].sort_values("Day", ascending=False)

In [ ]:
map_5_days = (df_day_map['Day'] >= 5).sum()
print(f"{map_5_days} mapping contributors on 5 days or more in {year} ({map_5_days/len(df_day_map):.1%})")

In [ ]:
df_day_valid = df[df["Type"] == "VALIDATION"][["Author", "Year", "Month", "Day"]].drop_duplicates()[["Author","Day"]].groupby("Author").count()
df_day_valid[df_day_valid["Day"] > 30].sort_values("Day", ascending=False)

In [ ]:
valid_5_days = (df_day_valid['Day'] >= 5).sum()
print(f"{valid_5_days} validators on 5 days or more in 2021 ({valid_5_days/len(df_day_valid):.1%})")

In [ ]:
df_day.sort_values("Day", ascending=False).reset_index().to_csv(os.path.join(str(year), "Contributor_by_contribution_days.csv"), index=None, encoding=encoding)

## Contributions

In [ ]:
total_dur = df['Duration'].sum() / 3600
total_map = df[df['Type'] == 'MAPPING']['Duration'].sum() / 3600
total_val = df[df['Type'] == 'VALIDATION']['Duration'].sum() / 3600
print(f"{total_dur:.5} hours of contributions ({total_dur/7:.4} working days, {total_dur/7/229:.3} ETP)")
print(f"{total_map:.5} hours of mapping ({total_map/7:.4} working days, {total_map/7/229:.3} ETP)")
print(f"{total_val:.5} hours of validation ({total_val/7:.4} working days, {total_val/7/229:.3} ETP)")

In [ ]:
print(f"{len(df['Project'].unique())} projects")

In [ ]:
validators = df[df['Type'] == 'VALIDATION']["Author"].unique()
mappers = df[df['Type'] == 'MAPPING']["Author"].unique()
print("Validators who are not mappers")
for validator in validators:
    if validator not in mappers:
        print(validator)

## Duration by project

In [ ]:
project_dur = df[["Project", "Duration"]].groupby("Project").sum()
project_dur["Duration_Hour"] = project_dur["Duration"] / 3600
project_dur["Duration_Working_Days"] = project_dur["Duration_Hour"] / 7
project_dur["Duration_Working_Days"] = project_dur["Duration_Hour"] / 7
project_dur.sort_values("Duration", ascending=False).head(10)

In [ ]:
project_dur.sort_values("Duration").head(10)

# Export key stats

In [ ]:
stats = {
    "contrib": len(g),
    "contrib_3_or_more_projects": int(more_than_three_projects),
    "map_contrib": len(g_map),
    "map_contrib_3_or_more_projects": int(map_more_than_three_projects),
    "valid_contrib": len(g_valid),
    "valid_contrib_3_or_more_projects": int(valid_more_than_three_projects),
    "contrib_5_days_or_more": int(contrib_5_days),
    "map_contrib_5_days_or_more": int(map_5_days),
    "valid_contrib_5_days_or_more": int(valid_5_days)
}

In [ ]:
with open(os.path.join(str(year), "stats.json"), "w") as f:
    json.dump(stats, f)